In [4]:
# Translated to .py by Meritxell Pacheco
# 2017
# Adapted to PandasBiogeme by Michel Bierlaire
# Sun Oct 21 23:15:31 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit

pandas = pd.read_table("lpmc02.dat")
database = db.Database("LPMC",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Exclude
# exclude = (  ArrivalTimeHours_1   ==  -1  )
# database.remove(exclude)
  
# Choice
chosenAlternative = travel_mode

#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1	 = Beta('Constant1',0,None,None,1)
Constant2	 = Beta('Constant2',0,None,None,0)
Constant3	 = Beta('Constant3',0,None,None,0)
Constant4	 = Beta('Constant4',0,None,None,0)
Cost	 = Beta('Cost',0,None,None,0)
Total_TT1	 = Beta('Total_TT1',0,None,None,0)
Total_TT2	 = Beta('Total_TT2',0,None,None,0)
Total_TT3	 = Beta('Total_TT3',0,None,None,0)
Total_TT4	 = Beta('Total_TT4',0,None,None,0)
StartTime1	 = Beta('StartTime1',0,None,None,0)
StartTime2	 = Beta('StartTime2',0,None,None,0)
StartTime3	 = Beta('StartTime3',0,None,None,0)
StartTime4	 = Beta('StartTime4',0,None,None,0)

# Define here arithmetic expressions for name that are not directly
# available from the data

dur_public = DefineVariable('dur_public', dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int ,database)
cost_public = DefineVariable('cost_public', cost_transit ,database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving

Opt1 = Constant1 + Total_TT1 * dur_walking + StartTime1 * start_time
Opt2 = Constant2 + Total_TT2 * dur_cycling + StartTime2 * start_time
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * dur_public + StartTime3 * start_time
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * dur_driving + StartTime4 * start_time
V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}

# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_lpmc_model2_StartTimeSpe"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

               Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
Constant2      -4.82    0.353   -13.7        0         0.402          -12   
Constant3      -2.42    0.214   -11.3        0         0.229        -10.6   
Constant4       -2.4    0.203   -11.8        0         0.229        -10.5   
Cost          -0.154   0.0151   -10.2        0        0.0148        -10.4   
StartTime1   -0.0153  0.00981   -1.57    0.117          0.01        -1.53   
StartTime2 -0.000649   0.0149 -0.0435    0.965        0.0166      -0.0391   
StartTime3   -0.0049  0.00738  -0.665    0.506       0.00768       -0.639   
StartTime4    0.0211  0.00687    3.07  0.00214       0.00723         2.92   
Total_TT1      -8.23    0.294     -28        0         0.457          -18   
Total_TT2       -5.9    0.444   -13.3        0          0.51        -11.6   
Total_TT3      -3.67    0.237   -15.5        0         0.243        -15.1   
Total_TT4      -6.59    0.335   -19.7        0         0.378        -17.5   